In [1]:
from pyspark.sql import SparkSession
import pandas as pd
import scipy as sc
from itertools import permutations
from scipy.sparse import coo_matrix
import numpy as np
from google.cloud import storage
import json
import scipy
import os

In [2]:
client = storage.Client()
bucket = client.bucket('bgse-datawarehousing-random-tweets')
paths = []
for blob in bucket.list_blobs(prefix=''):
    paths += ['gs://bgse-datawarehousing-random-tweets/'+blob.name]


In [3]:
def read_data(path_list):
    return spark.read.json(path_list)

In [4]:
def get_value_list(data_):
    a=data_.rdd \
        .filter(lambda l: l.entities is not None) \
        .filter(lambda x: len(x.entities.hashtags) > 1) \
        .map(lambda x: [i.text.lower() for i in x.entities.hashtags]) \
        .flatMap(lambda r: permutations(r,2)) \
        .map(lambda q: (q,1)) \
        .reduceByKey(lambda a,b:a+b ) \
        .collect()

    with open('list_of_values.json','w') as outfile:
        json.dump(a,outfile)
    return a

In [5]:
def get_sorted_unique_values_dict(val_list):
    sort_list = sorted(list(set([i[0][0] for i in val_list])))
    dict_of_words = {sort_list[i] : i for i in range(0,len(sort_list))}
    with open('col_names_dict.json','w') as outfile:
        json.dump(dict_of_words,outfile)
    return dict_of_words


In [6]:
def create_sparse_matrix(val_list,dict_cols):
    row  = np.array([dict_cols[i[0][0]] for i in val_list])
    col  = np.array([dict_cols[i[0][1]] for i in val_list])
    data = np.array([i[1] for i in val_list])
    return coo_matrix((data, (row, col)), shape=(len(dict_cols), len(dict_cols)))

In [7]:
def get_backup(col_names = 'col_names_dict.json',value_list = 'list_of_values.json'):
    with open(col_names, 'r') as infile:
        dict_col_names = json.load(infile)
    with open(value_list,'r') as infile:
        val_list = json.load(infile)
    return val_list,dict_col_names

In [ ]:
#def main_program():
data = read_data(paths)
print('data loaded')
value_list = get_value_list(data)
print('value_list created')
dict_columns = get_sorted_unique_values_dict(value_list)
print('dict_columns created')
value_list, dict_columns = get_backup()
print('Read from backup')
sparse_mat = create_sparse_matrix(value_list,dict_columns)
scipy.sparse.save_npz('sparse_matrix.npz', sparse_mat)
#return sparse_mat

data loaded
value_list created
dict_columns created
Read from backup


In [9]:
reloaded_matrix = scipy.sparse.load_npz('sparse_matrix.npz')

In [10]:
reloaded_matrix


<260019x260019 sparse matrix of type '<type 'numpy.int64'>'
	with 2561237 stored elements in COOrdinate format>